# Tensor与基本操作

In [2]:
import torch
import numpy as np

## 1. Tensor创建与初始化

### 直接创建

In [3]:
l1 = [[1, 2],[3, 4]]
s1 = torch.tensor(l1)
print(s1)

tensor([[1, 2],
        [3, 4]])


### 随机创建

In [4]:
tensor_random = torch.randn(2, 3)
print(tensor_random)

tensor([[-0.9256,  1.1340,  0.1422],
        [-0.4552, -0.9432,  0.0841]])


### 全0/1张量

In [5]:
ones = torch.ones(2, 3) # 输入形状
zeros = torch.zeros(1, 4) # 同样输入形状
print(ones)
print("----------------------------")
print(zeros)

tensor([[1., 1., 1.],
        [1., 1., 1.]])
----------------------------
tensor([[0., 0., 0., 0.]])


### 从NumPy数组创建

In [6]:
np_arr = np.array([[1, 2], [3, 4]])
tensor_from_np = torch.from_numpy(np_arr)
print(tensor_from_np)

tensor([[1, 2],
        [3, 4]])


### 从另一个tensor创建

In [7]:
s2 = torch.ones_like(s1)
print(s2)

tensor([[1, 1],
        [1, 1]])


## 2. Tensor的属性
张量属性描述了它们的形状、数据类型以及存储它们的设备

In [8]:
tensor_attri  = torch.rand(3,4)

print(f"Shape of tensor: {tensor_attri.shape}")
print(f"Datatype of tensor: {tensor_attri.dtype}")
print(f"Device tensor is stored on: {tensor_attri.device}")

Shape of tensor: torch.Size([3, 4])
Datatype of tensor: torch.float32
Device tensor is stored on: cpu


## 3. Tensor的基本操作

### 标准的类 NumPy 索引和切片

In [9]:
tensor = torch.ones(4, 4)
print(f"First row: {tensor[0]}")
print(f"First column: {tensor[:, 0]}")
print(f"Last column: {tensor[..., -1]}")
tensor[:,1] = 0
print(tensor)

First row: tensor([1., 1., 1., 1.])
First column: tensor([1., 1., 1., 1.])
Last column: tensor([1., 1., 1., 1.])
tensor([[1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.],
        [1., 0., 1., 1.]])


### cat和stack

### item()
`item()`是 `tensor` 的一个方法，用于将包含单个元素的 `tensor` 转换为 Python `标量`（如 int、float 等）。它的核心作用是提取 `tensor` 中唯一的元素值，方便在 Python 环境中使用（如打印、计算、保存等）。

In [31]:
mat_item_pre = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8]]).sum
print(mat_item_pre())


tensor(36)


In [ ]:
mat_item = mat_item_pre().item()
print(mat_item)
# mat_item_pre 是一个方法对象的引用
# 它指向 sum 这个方法本身，但没有执行
# 类似于 C++ 中的函数指针或 Python 中的可调用对象


36


### tensor的形状变换

#### 1. view


In [10]:
# view的本质是 “重新解读” tensor 在内存中的存储方式，它不会修改 tensor 的元素本身，只会改变对元素的维度划分。
mat = torch.tensor([[1, 2], [3, 4]])
mat2 = mat.view(1, 4)
print(f"mat:\n{mat}")
print(f"mat2:\n{mat2}")

mat:
tensor([[1, 2],
        [3, 4]])
mat2:
tensor([[1, 2, 3, 4]])


In [11]:
# view中可以用-1表示 “该维度的大小由其他维度和总元素数自动计算”，无需手动计算
mat3 = mat.view(4, -1)
print(f"mat3:\n{mat3}")

mat3:
tensor([[1],
        [2],
        [3],
        [4]])


注意：view操作要求原 tensor 必须是在内存中连续存储，否则会直接报错。

#### 2. reshape

reshape的核心作用与view一致：在不改变元素总数的前提下，重新定义 tensor 的维度结构。用法也一样

但是reshape同时兼容了在内存中连续存储的tensor和非连续存储的tensor
当原 tensor 是连续的：reshape的行为与view几乎一致，返回的是原 tensor 的 “视图”（共享内存，修改会相互影响）。
当原 tensor 是非连续的：reshape会先自动创建一个连续的副本（复制数据），再重塑形状，返回的是副本（不共享内存，修改互不影响）。


#### 3. permute
permute的作用是按照指定的顺序重新排列 tensor 的维度，例如将一个 3 维 tensor 的维度(0,1,2)调整为(2,0,1)，从而改变各维度的含义（如从 “通道 - 高 - 宽” 转为 “宽 - 通道 - 高”）

In [12]:
mat = torch.arange(3 * 4).view(1, 3, 4)
print(mat.shape)
print(mat)

torch.Size([1, 3, 4])
tensor([[[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]])


In [13]:
mat_per = mat.permute(0, 2, 1)
print(mat_per.shape)
print(mat_per)

torch.Size([1, 4, 3])
tensor([[[ 0,  4,  8],
         [ 1,  5,  9],
         [ 2,  6, 10],
         [ 3,  7, 11]]])


permute不会修改 tensor 的元素值，只会改变维度的逻辑排列顺序，因此：
- 元素总数不变（新 shape 的各维度乘积与原 shape 相同）；
- 返回的是原 tensor 的视图（view），与原 tensor 共享内存（修改视图会影响原 tensor）。

注意:permute会导致 tensor 非连续

#### 4. transpose
transpose的核心作用是将 tensor 中两个指定维度的位置互换，从而改变 tensor 的形状（但元素总数不变）。例如，对于一个 3 维 tensor (a, b, c)，交换维度 1 和 2 后，形状会变为(a, c, b)。

```py
# 函数式调用
torch.transpose(input, dim0, dim1)

# 方法式调用（更常用）
tensor.transpose(dim0, dim1)
```

`dim0`和`dim1`：需要交换的两个维度的索引（从 0 开始），必须是合法的维度索引（例如，2 维 tensor 只能交换 0 和 1）

#### 5. squeeze和unsqueeze
squeeze：移除大小为 1 的维度
- squeeze的作用是删除 tensor 中所有（或指定）大小为 1 的维度，使 tensor 的形状更简洁（维度数量减少）。

```py
# 移除所有大小为1的维度
tensor.squeeze()

# 只移除指定维度（若该维度大小为1）
tensor.squeeze(dim)
```
- `dim`：可选参数，指定要移除的维度索引（从 0 开始）。若该维度大小不为 1，操作无效（不会报错，返回原 tensor）。


unsqueeze：插入大小为 1 的维度
- unsqueeze的作用是在指定位置插入一个大小为 1 的新维度，使 tensor 的维度数量增加（常用于给 tensor “补维度” 以匹配模型输入要求）。
  
```py
tensor.unsqueeze(dim)
```

- `dim`：必选参数，指定插入新维度的位置（索引），范围为[-n-1, n]（n为原 tensor 的维度数），负数表示从后往前数（例如dim=-1表示插入到最后一个维度之后）。

## autograd
创建Tensor的时候有一个参数需要注意`requires_grad=True`, 这个参数可以追踪这个变量，从而便于后期求导(反向传播时用到)

In [14]:
# 创建张量
x = torch.tensor([[1., 2.], [3., 4.]], requires_grad=True)
y = torch.tensor([[2., 2.], [2., 2.]])
z = x * y + 3
out = z.mean()

这里x中的参数`requires_grad=True`, 所以可以看成变量了， 所以这里的计算过程就是
$$
x = \left(\begin{matrix}
x_{11} & x_{12} \\
x_{21} & x_{22} 
\end{matrix}
\right)
$$

$$
z = x * y + 3 = \left(
\begin{matrix}
2x_{11} + 3 & 2x_{12} + 3 \\
2x_{21} + 3 & 2x_{22} + 3
\end{matrix}
\right)
$$

$$
out = z.mean() = \left(
\begin{matrix}
\frac{2x_{11} + 3}{4} & \frac{2x_{12} + 3}{4} \\
\frac{2x_{21} + 3}{4} & \frac{2x_{22} + 3}{4}
\end{matrix}
\right)
$$

### 自动求导

In [15]:
# 自动求导
out.backward()

这里`out.backward()`是一个动作，这个函数的设计目的不是返回梯度值，而是计算梯度并将其存储在相应张量的`.grad`属性中。

当我们调用`out.backward()`的时候，PyTorch会：
- 从`out`开始，沿着计算图反向传播
- 计算`out`相对于所有`requires_grad=True`的叶子节点张量的梯度
- 将计算出的梯度累加到这些张量的`.grad`属性上

所以这里发生了这件事情

$$
\frac{\partial{out}}{\partial_{x_{ij}}} = \left(
\begin{matrix}
\frac{\partial_{\frac{2x_{11} + 3}{4}}}{\partial_{x_{11}}} & \frac{\partial_{\frac{2x_{12} + 3}{4}}}{\partial_{x_{12}}} \\
\frac{\partial_{\frac{2x_{21} + 3}{4}}}{\partial_{x_{21}}} & \frac{\partial_{\frac{2x_{22} + 3}{4}}}{\partial_{x_{22}}}
\end{matrix}
\right)
$$
最后结果就是$\frac{1}{2}$

In [16]:
print(x.grad)

tensor([[0.5000, 0.5000],
        [0.5000, 0.5000]])


### 练习：用autograd完成一个线性回归

目标：用PyTorch的 Tensor + autograd, 从随机数据中学习一个$y = 3x + 2$的关系出来

- step 1. 构造数据

In [20]:
torch.manual_seed(42)
x = torch.randn(100, 1)
y = 3 * x + 2 + 0.1 * torch.randn(100, 1) # 加一点小噪声

- step 2. 初始化参数

In [24]:
w = torch.randn(1, 1, requires_grad=True)
b = torch.zeros(1, requires_grad=True)
lr = 0.1

- step 3. 训练循环

In [ ]:
for step in range(200):

    # 前向传播
    y_pred = x @ w + b
    loss = ((y_pred - y) ** 2).mean()

    # 反向传播
    loss.backward()

    # 参数更新（在 no_grad 下操作数值）
    with torch.no_grad():
        w -= lr * w.grad
        b -= lr * b.grad

    # 梯度清零
    w.grad.zero_()
    b.grad.zero_()
    
    if step % 20 == 0:
        print(step, loss.item())
print("w:", w.item(), "b:", b.item())

0 14.791001319885254
20 0.009392197243869305
40 0.007803839631378651
60 0.007803605869412422
80 0.007803606800734997
100 0.007803606800734997
120 0.007803606800734997
140 0.007803606800734997
160 0.007803606800734997
180 0.007803606800734997
w: 3.001180410385132 b: 2.003567934036255
